In [1]:
!pip install langchain-groq

In [2]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0,
    max_retries=2,
    api_key='gsk...Vv'
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [4]:
!pip install chromadb

In [5]:
!pip install -U langchain-community

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-ii-finance-and-procurement-technology/job/R-60938")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Software Engineer II – Finance and Procurement Technology










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Men

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
chain_extract = prompt_extract | llm #pipe operation
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer II – Finance and Procurement Technology',
 'experience': {'required': 'AWS experience: EC2, Lambda, ECS, EKS, SQS, DynamoDB, S3, CLI',
  'desired': 'Working experience in integrating with SaaS and Open Source applications is a plus'},
 'skills': {'programming': ['Python', 'Java', 'Node.js'],
  'cloud': ['AWS'],
  'tools': ['Jenkins', 'GitHub', 'IntelliJ', 'Jira'],
  'architecture': ['microservices'],
  'testing': ['continuous integration', 'testing strategies']},
 'description': 'As a Software Engineer II, you will guide our Finance Integration Services squad in navigating technical design and implementation strategies. You will innovate, deliver high-quality code, share best practices and make critical technical design choices.'}

In [9]:
type(json_res)

dict

In [10]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

C:\Users\KIIT\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [01:59<00:00, 694kiB/s]   


In [26]:
# Flatten job['skills'] into a single list of keywords
query_keywords = []
for category in job['skills'].values():
    query_keywords.extend(category)

# Optional: Join the keywords into one string for better semantic querying
query_string = " ".join(query_keywords)

# Query the ChromaDB vector store
links = collection.query(query_texts=[query_string], n_results=2).get('metadatas', [])
print(links)

#links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
#links

[[{'links': 'https://example.com/devops-portfolio'}, {'links': 'https://example.com/python-portfolio'}]]


In [27]:
job = json_res
job['skills']

{'programming': ['Python', 'Java', 'Node.js'],
 'cloud': ['AWS'],
 'tools': ['Jenkins', 'GitHub', 'IntelliJ', 'Jira'],
 'architecture': ['microservices'],
 'testing': ['continuous integration', 'testing strategies']}

In [28]:
job

{'role': 'Software Engineer II – Finance and Procurement Technology',
 'experience': {'required': 'AWS experience: EC2, Lambda, ECS, EKS, SQS, DynamoDB, S3, CLI',
  'desired': 'Working experience in integrating with SaaS and Open Source applications is a plus'},
 'skills': {'programming': ['Python', 'Java', 'Node.js'],
  'cloud': ['AWS'],
  'tools': ['Jenkins', 'GitHub', 'IntelliJ', 'Jira'],
  'architecture': ['microservices'],
  'testing': ['continuous integration', 'testing strategies']},
 'description': 'As a Software Engineer II, you will guide our Finance Integration Services squad in navigating technical design and implementation strategies. You will innovate, deliver high-quality code, share best practices and make critical technical design choices.'}

In [29]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking Efficiency in Finance and Procurement Technology

Dear [Hiring Manager's Name],

I came across the job description for a Software Engineer II – Finance and Procurement Technology at [Company Name], and I was impressed by the requirements and expectations. As a Business Development Executive at AtliQ, an AI & Software Consulting company, I'd like to introduce you to our capabilities in fulfilling your needs.

AtliQ has a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in finance and procurement technology aligns perfectly with your requirements, and I'd like to highlight a few relevant projects from our portfolio:

* Our DevOps portfolio showcases our expertise in AWS, which aligns with your required experience in EC2, Lambda, ECS, EKS, SQS, DynamoDB, S3, and CLI. You can view our portfolio here: https://example.com/devops-portfolio
* Ou